<a href="https://colab.research.google.com/github/DMar35/machine-LEARNING/blob/main/linear_regression/PyTorchLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [ ]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert loaded dataset to numpy tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

# --- Reshape Target Tensors ---
# The model's output (from nn.Linear(..., 1)) has a 2D shape of [batch_size, 1].
# Our y tensors are currently 1D vectors of shape [batch_size].
# To prevent a shape mismatch error in the loss function, we reshape y to a 2D column vector.
# .view(-1, 1) tells PyTorch: "I want 1 column, and you figure out the number of rows (-1) needed."
# -1 is usually passed in as the second argument and {variable_name}.size(0) as the first
y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)



In [ ]:
class PyTorchLRModel(nn.Module):
  # NOTE: For linear regression, output_features is always 1.
  # It is simply added as a param here to show best practices.
  def __init__(self, input_features, output_features=1):
    super().__init__()
    self.linear = nn.Linear(input_features, output_features) # 1 represents number of output features

  def forward(self, data):
    return self.linear(data)

In [29]:
n_samples, n_features = X_train.shape
model = PyTorchLRModel(n_features)

LEARNING_RATE = 0.1
EPOCHS = 5000
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

# This training loop uses batch gradient descent(despite the SGD name lol)
# To use mini-batch GD or SGD, use DataLoader and update training loop to loop through each batch
for epoch in range(EPOCHS):
  # Compute predictions by passing training data to the model
  # Automatically calls the forward function
  prediction = model(X_train)

  # Loss function, MSE in this case
  loss = loss_function(prediction, y_train)

  # Compute gradients
  loss.backward()

  # Update weights
  optimizer.step()

  # Reset gradients to zero for next epoch
  optimizer.zero_grad()

  if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')


model.eval()
with torch.no_grad():
    train_loss = loss_function(model(X_train), y_train)
    test_loss = loss_function(model(X_test), y_test)

print("\n--- Evaluation ---")
print(f"Final Training Loss (MSE): {train_loss.item():.4f}")
print(f"Final Test Loss (MSE):     {test_loss.item():.4f}")


Epoch [500/5000], Loss: 4010.1594
Epoch [1000/5000], Loss: 3443.9465
Epoch [1500/5000], Loss: 3214.8474
Epoch [2000/5000], Loss: 3094.1084
Epoch [2500/5000], Loss: 3022.8921
Epoch [3000/5000], Loss: 2979.0203
Epoch [3500/5000], Loss: 2951.4006
Epoch [4000/5000], Loss: 2933.7334
Epoch [4500/5000], Loss: 2922.2607
Epoch [5000/5000], Loss: 2914.6968

--- Evaluation ---
Final Training Loss (MSE): 2914.6841
Final Test Loss (MSE):     2866.7810
